In [152]:
print ("============================================================")
print (" Course Work                              Robiul Islam      ")
print ("============================================================")


import pandas as pd 
from sklearn.utils import shuffle
from itertools import combinations
import bisect
import math

data = pd.read_csv("datafull.csv",sep=';') 
gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
gender = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/SocialAttributes(gender).csv')


data['Genaration']=gen
data['gender'] = gender
# Remove Null value
#data.dropna(inplace = True)
data.fillna(data.mean(), inplace=True)
data = shuffle(data, random_state=7)
print(data.head(10))
def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

def shapley_calc(N,characteristic_function,n):
    shapley_values = []
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                try:
                    cmod = len(j)
                    Cui = j[:]
                    l = N.index(j)
                    k = N.index(Cui)
                    temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                               float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                    shapley += temp
                except:
                    temp = float(N[0][0])
                    shapley += temp

        shapley_values.append(shapley)
    return shapley_values 

result =[]
for row in range(data.shape[0]):
    PS = power_set(data.iloc[row])
    Chan_fun =[]
    for i in PS:
        Chan_fun.append(max(i))
    Value = shapley_calc(PS,Chan_fun,5)
    result.append(max(Value))
    
data['shapley_value'] = result

def fitScaler(d):
    v = d.values
    return v / np.sqrt(np.sum(v**2))

data['separation'] = fitScaler(data['separation'])
data['work'] = fitScaler(data['work'])
data['partner'] = fitScaler(data['partner'])
data['marriage'] = fitScaler(data['marriage'])
data['children'] = fitScaler(data['children'])
data['shapley_value'] = fitScaler(data['shapley_value'])

print(data.head(10))

# gen = pd.read_csv('/Users/HSE/coursework/python_vmv/SeqDem/gen.csv')
# print(gen.head())

 Course Work                              Robiul Islam      
           work  separation    partner   marriage   children  Genaration  \
1627  13.000000   21.000000  23.058615  25.000000  36.000000           1   
1678  20.000000   23.000000  23.000000  22.000000  23.000000           1   
631   23.000000   18.000000  23.058615  25.000000  27.000000           2   
1940  17.000000   15.000000  19.000000  20.000000  24.000000           2   
2     19.193059   20.754376  23.058615  22.902162  24.075645           0   
1125  19.000000   20.000000  23.058615  26.000000  26.000000           5   
2138  18.000000   20.000000  23.058615  24.000000  24.000000           4   
737   17.000000   25.000000  23.058615  22.902162  24.075645           3   
832   17.000000   20.000000  20.000000  20.000000  18.000000           4   
204   16.000000   20.754376  16.000000  19.000000  19.000000           6   

      gender  
1627       0  
1678       1  
631        1  
1940       1  
2          0  
1125       0

# This code is 

In [162]:
import tensorflow as tf
import datetime, os

from tensorflow.python.keras.callbacks import TensorBoard 
from time import time

In [163]:

from keras.models import Sequential
from keras.layers import Dense
import numpy
import random
from sklearn.model_selection import train_test_split

# fix random seed for reproducibility
batch_size = 5
# load pima indians dataset
# split into input (X) and output (Y) variables
X = data[['work','separation','partner','marriage','children','shapley_value','gender']].values
#y = data['Genaration'].values

data.loc[data['Genaration'] <= 3, 'Genaration'] = 0
data.loc[data['Genaration'] >  3, 'Genaration'] = 1
y = data['Genaration'].values

# create model

score =[]
for i in range(10):
    random.seed(random.randint(1, 100))
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random.randint(1, 100))

    model = Sequential()
    model.add(Dense(12, input_dim=7, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(6, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
# Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
 #   logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = TensorBoard(log_dir="log/{}".format(time()))
    model.fit(X_train, y_train, epochs=100,batch_size=random.randint(1, 100),callbacks=[tensorboard_callback])
# evaluate the model
    s = model.evaluate(X_test, y_test)
    score.append(s)
    print("\n%s: %.2f%%" % (model.metrics_names[1], s[1]*100))

Epoch 1/100
1868/1868 [==============================] - 5s 3ms/step - loss: 0.5230 - acc: 0.9336
Epoch 2/100
1868/1868 [==============================] - 0s 183us/step - loss: 0.2374 - acc: 1.0000
Epoch 3/100
1868/1868 [==============================] - 0s 186us/step - loss: 0.0884 - acc: 1.0000
Epoch 4/100
1868/1868 [==============================] - 0s 187us/step - loss: 0.0355 - acc: 1.0000
Epoch 5/100
1868/1868 [==============================] - 0s 193us/step - loss: 0.0200 - acc: 1.0000
Epoch 6/100
1868/1868 [==============================] - 0s 201us/step - loss: 0.0132 - acc: 1.0000
Epoch 7/100
1868/1868 [==============================] - 0s 201us/step - loss: 0.0083 - acc: 1.0000
Epoch 8/100
1868/1868 [==============================] - 0s 208us/step - loss: 0.0070 - acc: 1.0000
Epoch 9/100
1868/1868 [==============================] - 0s 195us/step - loss: 0.0051 - acc: 1.0000
Epoch 10/100
1868/1868 [==============================] - 0s 223us/step - loss: 0.0038 - acc: 1.0000
E

1868/1868 [==============================] - 0s 197us/step - loss: 7.7946e-05 - acc: 1.0000
Epoch 80/100
1868/1868 [==============================] - 0s 185us/step - loss: 2.2049e-05 - acc: 1.0000
Epoch 81/100
1868/1868 [==============================] - 0s 179us/step - loss: 2.3176e-05 - acc: 1.0000
Epoch 82/100
1868/1868 [==============================] - 0s 179us/step - loss: 4.8274e-05 - acc: 1.0000
Epoch 83/100
1868/1868 [==============================] - 0s 181us/step - loss: 2.5694e-05 - acc: 1.0000
Epoch 84/100
1868/1868 [==============================] - 0s 180us/step - loss: 6.5769e-05 - acc: 1.0000
Epoch 85/100
1868/1868 [==============================] - 0s 188us/step - loss: 2.8902e-05 - acc: 1.0000
Epoch 86/100
1868/1868 [==============================] - 0s 184us/step - loss: 5.5977e-05 - acc: 1.0000
Epoch 87/100
1868/1868 [==============================] - 0s 181us/step - loss: 2.7932e-05 - acc: 1.0000
Epoch 88/100
1868/1868 [==============================] - 0s 186us/s

1868/1868 [==============================] - 0s 58us/step - loss: 9.4464e-04 - acc: 1.0000
Epoch 60/100
1868/1868 [==============================] - 0s 55us/step - loss: 8.6825e-04 - acc: 1.0000
Epoch 61/100
1868/1868 [==============================] - 0s 54us/step - loss: 8.5907e-04 - acc: 1.0000
Epoch 62/100
1868/1868 [==============================] - 0s 53us/step - loss: 8.4668e-04 - acc: 1.0000
Epoch 63/100
1868/1868 [==============================] - 0s 54us/step - loss: 8.7668e-04 - acc: 1.0000
Epoch 64/100
1868/1868 [==============================] - 0s 53us/step - loss: 6.3439e-04 - acc: 1.0000
Epoch 65/100
1868/1868 [==============================] - 0s 54us/step - loss: 6.2791e-04 - acc: 1.0000
Epoch 66/100
1868/1868 [==============================] - 0s 55us/step - loss: 7.1738e-04 - acc: 1.0000
Epoch 67/100
1868/1868 [==============================] - 0s 55us/step - loss: 7.0579e-04 - acc: 1.0000
Epoch 68/100
1868/1868 [==============================] - 0s 54us/step - loss

1868/1868 [==============================] - 0s 86us/step - loss: 0.0011 - acc: 1.0000
Epoch 39/100
1868/1868 [==============================] - 0s 83us/step - loss: 0.0010 - acc: 1.0000
Epoch 40/100
1868/1868 [==============================] - 0s 84us/step - loss: 0.0011 - acc: 1.0000
Epoch 41/100
1868/1868 [==============================] - 0s 83us/step - loss: 0.0010 - acc: 1.0000
Epoch 42/100
1868/1868 [==============================] - 0s 83us/step - loss: 7.9900e-04 - acc: 1.0000
Epoch 43/100
1868/1868 [==============================] - 0s 83us/step - loss: 8.8154e-04 - acc: 1.0000
Epoch 44/100
1868/1868 [==============================] - 0s 82us/step - loss: 8.5970e-04 - acc: 1.0000
Epoch 45/100
1868/1868 [==============================] - 0s 86us/step - loss: 6.8299e-04 - acc: 1.0000
Epoch 46/100
1868/1868 [==============================] - 0s 84us/step - loss: 7.5893e-04 - acc: 1.0000
Epoch 47/100
1868/1868 [==============================] - 0s 83us/step - loss: 6.6784e-04 - a

1868/1868 [==============================] - 0s 86us/step - loss: 0.0314 - acc: 1.0000
Epoch 18/100
1868/1868 [==============================] - 0s 86us/step - loss: 0.0255 - acc: 1.0000
Epoch 19/100
1868/1868 [==============================] - 0s 86us/step - loss: 0.0239 - acc: 1.0000
Epoch 20/100
1868/1868 [==============================] - 0s 86us/step - loss: 0.0214 - acc: 1.0000
Epoch 21/100
1868/1868 [==============================] - 0s 87us/step - loss: 0.0170 - acc: 1.0000
Epoch 22/100
1868/1868 [==============================] - 0s 88us/step - loss: 0.0162 - acc: 1.0000
Epoch 23/100
1868/1868 [==============================] - 0s 87us/step - loss: 0.0147 - acc: 1.0000
Epoch 24/100
1868/1868 [==============================] - 0s 87us/step - loss: 0.0135 - acc: 1.0000
Epoch 25/100
1868/1868 [==============================] - 0s 86us/step - loss: 0.0120 - acc: 1.0000
Epoch 26/100
1868/1868 [==============================] - 0s 96us/step - loss: 0.0116 - acc: 1.0000
Epoch 27/100


...

Epoch 36/100
1868/1868 [==============================] - 0s 186us/step - loss: 5.7352e-04 - acc: 1.0000
Epoch 37/100
1868/1868 [==============================] - 0s 188us/step - loss: 5.7129e-04 - acc: 1.0000
Epoch 38/100
1868/1868 [==============================] - 0s 189us/step - loss: 4.3212e-04 - acc: 1.0000
Epoch 39/100
1868/1868 [==============================] - 0s 187us/step - loss: 3.9177e-04 - acc: 1.0000
Epoch 40/100
1868/1868 [==============================] - 0s 184us/step - loss: 4.1024e-04 - acc: 1.0000
Epoch 41/100
1868/1868 [==============================] - 0s 187us/step - loss: 3.2203e-04 - acc: 1.0000
Epoch 42/100
1868/1868 [==============================] - 0s 183us/step - loss: 3.3551e-04 - acc: 1.0000
Epoch 43/100
1868/1868 [==============================] - 0s 188us/step - loss: 3.0722e-04 - acc: 1.0000
Epoch 44/100
1868/1868 [==============================] - 0s 183us/step - loss: 3.3741e-04 - acc: 1.0000
Epoch 45/100
1868/1868 [==============================]

1868/1868 [==============================] - 0s 57us/step - loss: 0.0427 - acc: 1.0000
Epoch 15/100
1868/1868 [==============================] - 0s 57us/step - loss: 0.0341 - acc: 1.0000
Epoch 16/100
1868/1868 [==============================] - 0s 57us/step - loss: 0.0262 - acc: 1.0000
Epoch 17/100
1868/1868 [==============================] - 0s 56us/step - loss: 0.0219 - acc: 1.0000
Epoch 18/100
1868/1868 [==============================] - 0s 57us/step - loss: 0.0179 - acc: 1.0000
Epoch 19/100
1868/1868 [==============================] - 0s 57us/step - loss: 0.0147 - acc: 1.0000
Epoch 20/100
1868/1868 [==============================] - 0s 56us/step - loss: 0.0128 - acc: 1.0000
Epoch 21/100
1868/1868 [==============================] - 0s 56us/step - loss: 0.0114 - acc: 1.0000
Epoch 22/100
1868/1868 [==============================] - 0s 57us/step - loss: 0.0094 - acc: 1.0000
Epoch 23/100
1868/1868 [==============================] - 0s 57us/step - loss: 0.0089 - acc: 1.0000
Epoch 24/100


1868/1868 [==============================] - 0s 57us/step - loss: 4.1360e-04 - acc: 1.0000
Epoch 95/100
1868/1868 [==============================] - 0s 57us/step - loss: 4.1791e-04 - acc: 1.0000
Epoch 96/100
1868/1868 [==============================] - 0s 57us/step - loss: 3.3569e-04 - acc: 1.0000
Epoch 97/100
1868/1868 [==============================] - 0s 57us/step - loss: 4.1805e-04 - acc: 1.0000
Epoch 98/100
1868/1868 [==============================] - 0s 57us/step - loss: 3.8266e-04 - acc: 1.0000
Epoch 99/100
1868/1868 [==============================] - 0s 57us/step - loss: 4.2638e-04 - acc: 1.0000
Epoch 100/100
468/468 [==============================] - 2s 4ms/step

acc: 100.00%
Epoch 1/100
1868/1868 [==============================] - 8s 4ms/step - loss: 0.5001 - acc: 0.9963
Epoch 2/100
1868/1868 [==============================] - 0s 117us/step - loss: 0.3681 - acc: 1.0000
Epoch 3/100
1868/1868 [==============================] - 0s 102us/step - loss: 0.2589 - acc: 1.0000
Epoch 4/

1868/1868 [==============================] - 0s 102us/step - loss: 2.8497e-04 - acc: 1.0000
Epoch 74/100
1868/1868 [==============================] - 0s 101us/step - loss: 2.0351e-04 - acc: 1.0000
Epoch 75/100
1868/1868 [==============================] - 0s 116us/step - loss: 3.0291e-04 - acc: 1.0000
Epoch 76/100
1868/1868 [==============================] - 0s 98us/step - loss: 2.0861e-04 - acc: 1.0000
Epoch 77/100
1868/1868 [==============================] - 0s 105us/step - loss: 1.8625e-04 - acc: 1.0000
Epoch 78/100
1868/1868 [==============================] - 0s 105us/step - loss: 1.9213e-04 - acc: 1.0000
Epoch 79/100
1868/1868 [==============================] - 0s 101us/step - loss: 2.1963e-04 - acc: 1.0000
Epoch 80/100
1868/1868 [==============================] - 0s 117us/step - loss: 1.4924e-04 - acc: 1.0000
Epoch 81/100
1868/1868 [==============================] - 0s 118us/step - loss: 1.8765e-04 - acc: 1.0000
Epoch 82/100
1868/1868 [==============================] - 0s 111us/st

1868/1868 [==============================] - 0s 196us/step - loss: 3.2180e-05 - acc: 1.0000
Epoch 52/100
1868/1868 [==============================] - 0s 201us/step - loss: 2.7058e-05 - acc: 1.0000
Epoch 53/100
1868/1868 [==============================] - 0s 200us/step - loss: 3.6077e-05 - acc: 1.0000
Epoch 54/100
1868/1868 [==============================] - 0s 198us/step - loss: 2.2974e-05 - acc: 1.0000
Epoch 55/100
1868/1868 [==============================] - 0s 194us/step - loss: 3.2381e-05 - acc: 1.0000
Epoch 56/100
1868/1868 [==============================] - 0s 195us/step - loss: 1.8694e-05 - acc: 1.0000
Epoch 57/100
1868/1868 [==============================] - 0s 198us/step - loss: 1.2679e-05 - acc: 1.0000
Epoch 58/100
1868/1868 [==============================] - 0s 196us/step - loss: 1.6911e-05 - acc: 1.0000
Epoch 59/100
1868/1868 [==============================] - 0s 220us/step - loss: 2.8002e-05 - acc: 1.0000
Epoch 60/100
1868/1868 [==============================] - 0s 192us/s

In [164]:
tensorboard_callback --logdir = log/

SyntaxError: invalid syntax (<ipython-input-164-f40e4ed6493c>, line 1)

In [126]:

import numpy
import pandas
from keras.layers import Dense, Dropout, Activation
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [127]:


X = data[['work','separation','partner','marriage','children','shapley_value','gender']].values.astype(float)
Y = data['Genaration'].values

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(200, input_dim=7, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(150, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(100, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(25, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model


estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=random.randint(1, 100), verbose=0)
kfold = KFold(n_splits=5, shuffle=True, random_state=random.randint(1, 100))
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Baseline: 24.74% (1.34%)


In [123]:
encoded_Y

array([1, 1, 2, ..., 4, 0, 5])

In [117]:
type(score)
score= np.array(score)
print(np.mean(score[:,1]*100))
score

52.628205179149276


array([[0.68964227, 0.53632479],
       [0.69357195, 0.51709402],
       [0.69010232, 0.53632479],
       [0.68939763, 0.53632479],
       [0.69237308, 0.52991453],
       [0.69279905, 0.53205128],
       [0.69092396, 0.52777778],
       [0.69996116, 0.48504274],
       [0.68893043, 0.55555556],
       [0.69384982, 0.50641026]])

In [ ]:
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=7, activation='relu'))
	model.add(Dense(8, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

In [21]:
import shap
import numpy as np

# select a set of background examples to take an expectation over
background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]

# explain predictions of the model on four images
e = shap.DeepExplainer(model, background)
# ...or pass tensors directly
# e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
shap_values = e.shap_values(X_train)
shap_values = np.array(shap_values)


In [22]:
shap_values[0]

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [1]:
print('hello world')

hello world


In [4]:
import lutorpy as lua
x =  torch.Tensor([[1, 2, 3], [1, 2, 3]]).view(-1, 2)
y =  torch.Tensor([[2, 1]]).view(2, -1)

ModuleNotFoundError: No module named 'lutorpy'

In [13]:
data.loc[data['Generation'] <= 3, 'Generation'] = 0
data.loc[data['Generation'] >  3, 'Generation'] = 1

In [14]:
data['Generation']

283     0
1329    0
2051    0
450     0
1135    0
1398    0
237     0
702     0
1037    0
913     0
505     0
2286    0
1679    0
1971    0
526     0
659     0
1247    0
1221    0
841     0
1106    0
20      0
1677    0
675     0
1601    0
463     0
1579    0
1286    0
603     0
155     0
1891    0
       ..
1585    0
1013    0
1953    0
306     0
1605    0
1561    0
1936    0
827     0
1483    0
1130    0
1575    0
353     0
2017    0
35      0
1933    0
539     0
0       0
481     0
1285    0
1730    0
342     0
513     0
207     0
1526    0
733     0
1674    0
476     0
214     0
1587    0
1383    0
Name: Generation, Length: 309, dtype: int64

In [20]:
data.loc[data['Generation'] <= 3, 'Generation'] = 0
data.loc[data['Generation'] >  3, 'Generation'] = 1
y = data['Genaration'].values
print(y)

[6 4 5 4 2 0 5 5 6 5 0 4 4 4 4 1 6 6 4 3 6 2 6 5 6 6 5 6 4 6 3 6 5 2 6 5 6
 0 4 3 4 4 4 5 4 5 0 0 3 5 4 3 5 3 4 1 6 6 2 3 3 5 6 1 3 4 4 6 4 5 4 4 4 5
 4 2 1 5 6 6 6 4 5 3 5 6 6 4 4 5 5 5 5 4 6 6 4 3 4 4 6 6 5 3 5 6 3 3 2 0 2
 5 3 3 4 4 0 4 3 4 2 1 0 5 5 3 6 0 5 6 3 3 6 5 4 6 5 2 5 5 6 5 4 6 2 2 5 4
 5 5 0 3 6 3 5 0 4 5 5 0 3 4 5 4 5 5 6 4 5 6 2 4 6 5 5 1 0 5 4 3 5 5 5 3 3
 6 5 0 6 4 1 5 5 6 6 6 6 6 4 6 2 6 4 4 4 3 5 0 4 2 3 6 3 6 6 6 6 4 6 3 4 4
 5 2 2 6 5 6 2 2 5 3 5 6 2 6 5 5 6 4 4 2 5 6 3 3 6 5 6 4 5 6 5 6 4 5 2 6 5
 0 3 6 6 5 5 2 6 0 4 3 6 3 4 2 4 4 5 5 5 0 4 4 5 2 4 5 6 6 5 2 0 3 2 5 4 6
 3 2 5 4 5 5 3 2 2 5 5 0 5]


In [25]:
X,y

(array([[ 1.76041518e+00, -3.91816240e-01,  5.56073510e-01,
         -2.51630386e+00, -1.16400797e+00,  6.36583409e-01,
         -8.06184817e-01, -1.83400197e-01, -1.78791289e+00,
         -1.15498263e+00,  5.10937774e-01,  1.44287693e+00,
          1.81038744e-01,  1.47016034e+00,  9.66539250e-01,
          3.82715174e-01,  1.62091229e+00,  1.89293198e-01,
          1.07343294e-01,  5.13505480e-01, -1.51045638e+00,
         -2.75293863e-02,  1.28376990e-01, -7.99547491e-01,
         -2.22414029e-01,  5.48129585e-01, -1.47656266e+00,
          1.74643509e+00, -6.74100249e-02, -1.56573815e+00,
          3.95860270e+00, -5.39681562e-01,  1.71066184e+00,
         -6.04602969e-01,  1.69380911e+00, -2.91594596e-01,
          1.01120706e+00,  5.57439453e-01, -1.11343610e+00,
         -7.31530982e-01, -1.63744959e+00, -2.51674208e-01,
          7.13049050e-01, -4.24492791e-01,  1.30620607e+00,
         -3.09121319e-01,  9.86352180e-01,  1.85300949e+00,
         -2.58982853e-01,  3.69190470e-0

In [61]:
X

NameError: name 'X' is not defined

In [75]:
print("Hello")

Hello


In [93]:
import pandas as pd 
data = pd.read_csv("allgenarationdata.csv") 
data.fillna(data.mean(), inplace=True)
print(data.head(20))

import numpy as np


from sklearn import preprocessing
# def fitScaler(d):
#     x = d.values
#     min_max_scaler = preprocessing.MinMaxScaler()
#     x_scaled = min_max_scaler.fit_transform(x)
#     return pd.DataFrame(x_scaled)

def fitScaler(d):
    v = d.values
    return v / np.sqrt(np.sum(v**2))

data['separation'] = fitScaler(data['separation'])
data['work'] = fitScaler(data['work'])
data['partner'] = fitScaler(data['partner'])
data['marriage'] = fitScaler(data['marriage'])
data['children'] = fitScaler(data['children'])


    

import n2w
wekagen = []
for i in data['generation']:
    tem = n2w.convert(i)
    wekagen.append(tem)
    
data["wekagen"] = wekagen


def power_set(List):
    PS = [list(j) for i in range(len(List)) for j in combinations(List, i+1)]
    return PS

def shapley_calc(N,characteristic_function,n):
    shapley_values = []
    for i in range(n):
        shapley = 0
        for j in N:
            if i not in j:
                try:
                    cmod = len(j)
                    Cui = j[:]
                    l = N.index(j)
                    k = N.index(Cui)
                    temp = float(float(characteristic_function[k]) - float(characteristic_function[l])) *\
                               float(math.factorial(cmod) * math.factorial(n - cmod - 1)) / float(math.factorial(n))
                    shapley += temp
                except:
                    temp = float(N[0][0])
                    shapley += temp

        shapley_values.append(shapley)
    return shapley_values 

result =[]
for row in range(data.shape[0]):
    PS = power_set(data.iloc[row])
    Chan_fun =[]
    
    for i in PS:
        Chan_fun.append(max(i))
    Value = shapley_calc(PS,Chan_fun,n)
    result.append(max(Value))
    
data['shapley_value'] = result

data.head()
#data.to_csv('allgenarationdata_normalized.csv')


# x = data[['separation','work','partner','marriage','children']].values.astype(float)
# min_max_scaler = preprocessing.MinMaxScaler()
# x_scaled = min_max_scaler.fit_transform(x)
# data_norm = pd.DataFrame(x_scaled)

# data['separation'] = data_norm[0]
# data['work'] = data_norm[1]
# data['partner'] = data_norm[2]
# data['marriage'] = data_norm[3]
# data['children'] = data_norm[4]

# data.dropna(inplace = True)
# data.info()

         work  separation    partner   marriage   children  gender  generation
0   19.000000   19.000000  20.000000  21.000000  29.000000       1           6
1   22.000000   28.000000  23.058615  22.902162  24.075645       1           6
2   19.193059   20.754376  23.058615  22.902162  24.075645       0           0
3   14.000000   17.000000  23.058615  18.000000  23.000000       1           3
4   18.000000   17.000000  35.000000  21.000000  22.000000       0           5
5   19.000000   37.000000  23.058615  20.000000  21.000000       1           2
6   19.000000   21.000000  23.058615  21.000000  21.000000       1           2
7   17.000000   18.000000  20.000000  22.902162  31.000000       0           5
8   21.000000   20.000000  23.058615  22.000000  23.000000       1           2
9   18.000000   25.000000  25.000000  25.000000  17.000000       1           3
10  22.000000   32.000000  23.058615  33.000000  37.000000       0           4
11  19.193059   20.754376  23.058615  22.902162  24.

TypeError: '<' not supported between instances of 'numpy.ndarray' and 'str'

In [94]:
data.shape[0]

2336

,work,separation,partner,marriage,children,gender,generation,wekagen
0,0.055672,0.050917,0.048595,0.051195,0.069056,1,6,six
4,0.052742,0.045558,0.085041,0.051195,0.052387,0,5,five
9,0.052742,0.066996,0.060744,0.060946,0.040481,1,3,three
20,0.067393,0.050917,0.046165,0.046319,0.047625,1,6,six
25,0.067393,0.061637,0.055884,0.056071,0.054768,0,6,six


In [59]:
data.to_csv('forweka.csv')

![](wekascrenshort/1.png)
![](wekascrenshort/2.png)

,work,separation,partner,marriage,children,gender,generation,wekagen
0,0.055672,0.050917,0.048595,0.051195,0.069056,1,6,six
4,0.052742,0.045558,0.085041,0.051195,0.052387,0,5,five
9,0.052742,0.066996,0.060744,0.060946,0.040481,1,3,three
20,0.067393,0.050917,0.046165,0.046319,0.047625,1,6,six
25,0.067393,0.061637,0.055884,0.056071,0.054768,0,6,six


In [71]:
import shap
import numpy as np
data = pd.read_csv("allgenarationdata.csv") 
data.dropna(inplace = True)

# select a set of background examples to take an expectation over
X_train = data[['work','separation','partner','marriage','children']]



background = X_train[np.random.choice(X_train.shape[0], 100, replace=False)]

# # explain predictions of the model on four images
# e = shap.DeepExplainer(model, background)
# # ...or pass tensors directly
# # e = shap.DeepExplainer((model.layers[0].input, model.layers[-1].output), background)
# shap_values = e.shap_values(X_train)
# shap_values = np.array(shap_values)
# shap_values


KeyError: '[217 223  93 108 284 235  98 216 256   6 131 237 228  24 161  12  16  30\n 222 261  71 269  91 297 145 100 262 169  50 305 204  97  37  34 124 246\n 281 293 303 163 307  90  28 299 265  69 101 214 267 255  95 224 141 189\n  53 125 103 215 207  68 102 221  75 172  18 122 274 213 111  27  25  62\n 176 266 260 257 175 236 232 183  11 263 271   1  79  60 127  15 139  38\n 117  77  52 202 276 140  74 130  84   9] not in index'

In [5]:
from sklearn.preprocessing import MinMaxScaler
column_names_to_normalize = ['work']
x = data[column_names_to_normalize].values
x_scaled = min_max_scaler.fit_transform(x)
df_temp = pd.DataFrame(x_scaled, columns=column_names_to_normalize, index = df.index)
data[column_names_to_normalize] = df_temp

TypeError: fit_transform() missing 1 required positional argument: 'X'

In [9]:






print("hello")
print(fitScaler([[-1, 2], [-0.5, 6], [0, 10], [1, 18]]))




hello
[[0.   0.  ]
 [0.25 0.25]
 [0.5  0.5 ]
 [1.   1.  ]]


In [47]:
!pip install n2w

  Running setup.py bdist_wheel for n2w ... done
  Stored in directory: /Users/robiulislam/Library/Caches/pip/wheels/2f/aa/b1/f591ac737d835491b07fb62ce6a041357fd8777de2942767a1
Successfully built n2w


In [48]:
import n2w
print(n2w.convert(10))

ten
